In [1]:
pip install sklearn_crfsuite

     |████████████████████████████████| 757kB 4.9MB/s 


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [3]:
pip install eli5

     |████████████████████████████████| 112kB 3.4MB/s 


In [4]:
import nltk
import sklearn_crfsuite
import eli5

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
import spacy
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
nlp = spacy.load("en_core_web_sm") 

In [11]:
train = pd.read_csv('task2_train.csv')
test = pd.read_csv('task2_test.csv')
dev = pd.read_csv('task2_dev.csv')
train.head()
dev.head()
test.head()

,Subject,Sentence,Label
0,0.deft,"However , those fields of science related to ...",O O B-Definition I-Definition I-Definiti...
1,0.deft,Deductive reasoning is a form of logical thin...,B-Term I-Term O B-Definition I-Definitio...
2,0.deft,A variable is any part of the experiment that...,B-Term I-Term O B-Definition I-Definitio...
3,0.deft,The control group contains every feature of t...,B-Term I-Term I-Term B-Definition I-Defi...
4,0.deft,"14 . In contrast , applied science or “ techn...",O O O O O B-Term I-Term O B-Alias-Te...


,Subject,Sentence,Label
0,0.deft,The scientific method is a method of research...,B-Term I-Term I-Term O B-Definition I-D...
1,0.deft,An ecosystem consists of all the living thing...,B-Term I-Term B-Definition I-Definition ...
2,0.deft,At the highest level of organization ( [ link...,O O O O O O O O O O O O B-Term ...
3,0.deft,125 . Glycogen is the storage form of glucose...,O O B-Term O B-Definition I-Definition ...
4,0.deft,The nucleotides combine with each other to fo...,B-Definition I-Definition I-Definition I-...


,Subject,Sentence,Label
0,biology,26 . Living things are highly organized and s...,NaN
1,biology,The atom is the smallest and most fundamental...,NaN
2,biology,It consists of a nucleus surrounded by electr...,NaN
3,biology,"47 . At its most fundamental level , life is ...",NaN
4,biology,Matter is any substance that occupies space a...,NaN


In [12]:
train['Sentence'] = train['Sentence'].apply(lambda x : x.strip())
dev['Sentence'] = dev['Sentence'].apply(lambda x: x.strip())
test['Sentence'] = test['Sentence'].apply(lambda x:x.strip())
train.head()
dev.head()

,Subject,Sentence,Label
0,0.deft,"However , those fields of science related to t...",O O B-Definition I-Definition I-Definiti...
1,0.deft,Deductive reasoning is a form of logical think...,B-Term I-Term O B-Definition I-Definitio...
2,0.deft,A variable is any part of the experiment that ...,B-Term I-Term O B-Definition I-Definitio...
3,0.deft,The control group contains every feature of th...,B-Term I-Term I-Term B-Definition I-Defi...
4,0.deft,"14 . In contrast , applied science or “ techno...",O O O O O B-Term I-Term O B-Alias-Te...


,Subject,Sentence,Label
0,0.deft,The scientific method is a method of research ...,B-Term I-Term I-Term O B-Definition I-D...
1,0.deft,An ecosystem consists of all the living things...,B-Term I-Term B-Definition I-Definition ...
2,0.deft,At the highest level of organization ( [ link ...,O O O O O O O O O O O O B-Term ...
3,0.deft,125 . Glycogen is the storage form of glucose ...,O O B-Term O B-Definition I-Definition ...
4,0.deft,The nucleotides combine with each other to for...,B-Definition I-Definition I-Definition I-...


In [0]:
def pos_tagger(df):
  df_pos_tags = []
  for i, val in df.iterrows():
    doc = nlp(df['Sentence'][i])
    pos_tags = []
    for token in doc:
      pos_tags.append(token.pos_)
    pos_tags = ' '.join(pos_tags)
    df_pos_tags.append(pos_tags)
  df['POS']= df_pos_tags
pos_tagger(train)
pos_tagger(dev)
pos_tagger(test)

In [14]:
train.head()

,Subject,Sentence,Label,POS
0,0.deft,"However , those fields of science related to t...",O O B-Definition I-Definition I-Definiti...,ADV PUNCT DET NOUN ADP NOUN VERB ADP DET ADJ N...
1,0.deft,Deductive reasoning is a form of logical think...,B-Term I-Term O B-Definition I-Definitio...,ADJ NOUN VERB DET NOUN ADP ADJ NOUN DET VERB D...
2,0.deft,A variable is any part of the experiment that ...,B-Term I-Term O B-Definition I-Definitio...,DET NOUN VERB DET NOUN ADP DET NOUN DET VERB V...
3,0.deft,The control group contains every feature of th...,B-Term I-Term I-Term B-Definition I-Defi...,DET NOUN NOUN VERB DET NOUN ADP DET ADJ NOUN A...
4,0.deft,"14 . In contrast , applied science or “ techno...",O O O O O B-Term I-Term O B-Alias-Te...,NUM PUNCT ADP NOUN PUNCT VERB NOUN CCONJ NOUN ...


In [24]:
test['Label'] = test['Sentence'].apply(lambda x: ' '.join(['O']*len(x.split())))
test['Label']

0      O O O O O O O O O O O O O O O O O O O O O O O O O
1                                O O O O O O O O O O O O
2                                      O O O O O O O O O
3                          O O O O O O O O O O O O O O O
4                                  O O O O O O O O O O O
                             ...                        
851        O O O O O O O O O O O O O O O O O O O O O O O
852    O O O O O O O O O O O O O O O O O O O O O O O ...
853                                          O O O O O O
854                  O O O O O O O O O O O O O O O O O O
855    O O O O O O O O O O O O O O O O O O O O O O O ...
Name: Label, Length: 856, dtype: object

In [0]:
def data_prep(df):
  data = []
  for i in range(df.shape[0]):
      # Take the word, POS tag, and its label
      words= df['Sentence'][i].split()
      pos =df['POS'][i].split()
      label = df['Label'][i].split()
      data.append([(w, pos, label) for (w, pos, label) in zip(words, pos, label)])
  return data
train_prep = data_prep(train)
dev_prep = data_prep(dev)
test_prep = data_prep(test)

In [26]:
test_prep[0]

[('26', 'NUM', 'O'),
 ('.', 'PUNCT', 'O'),
 ('Living', 'VERB', 'O'),
 ('things', 'NOUN', 'O'),
 ('are', 'VERB', 'O'),
 ('highly', 'ADV', 'O'),
 ('organized', 'ADJ', 'O'),
 ('and', 'CCONJ', 'O'),
 ('structured', 'ADJ', 'O'),
 (',', 'PUNCT', 'O'),
 ('following', 'VERB', 'O'),
 ('a', 'DET', 'O'),
 ('hierarchy', 'NOUN', 'O'),
 ('that', 'DET', 'O'),
 ('can', 'VERB', 'O'),
 ('be', 'VERB', 'O'),
 ('examined', 'VERB', 'O'),
 ('on', 'ADP', 'O'),
 ('a', 'DET', 'O'),
 ('scale', 'NOUN', 'O'),
 ('from', 'ADP', 'O'),
 ('small', 'ADJ', 'O'),
 ('to', 'ADP', 'O'),
 ('large', 'ADJ', 'O'),
 ('.', 'PUNCT', 'O')]

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_prep]
y_train = [sent2labels(s) for s in train_prep]

X_test = [sent2features(s) for s in dev_prep]
y_test = [sent2labels(s) for s in dev_prep]

In [28]:
X_train[0][1]

{'+1:postag': 'DET',
 '+1:postag[:2]': 'DE',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'those',
 '-1:postag': 'ADV',
 '-1:postag[:2]': 'AD',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.lower()': 'however',
 'bias': 1.0,
 'postag': 'PUNCT',
 'postag[:2]': 'PU',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': ',',
 'word[-3:]': ','}

In [29]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 44.9 s, sys: 39.9 ms, total: 45 s
Wall time: 45 s


In [0]:
# eli5.show_weights(crf, top=30)

In [31]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-Definition',
 'I-Definition',
 'B-Term',
 'I-Term',
 'B-Alias-Term',
 'I-Alias-Term',
 'B-Secondary-Definition',
 'I-Secondary-Definition',
 'B-Referential-Definition',
 'I-Referential-Definition',
 'B-Qualifier',
 'I-Qualifier',
 'B-Referential-Term',
 'B-Definition-frag',
 'I-Definition-frag',
 'I-Referential-Term',
 'B-Term-frag',
 'B-Alias-Term-frag',
 'I-Term-frag']

In [32]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.7840947911015923

In [0]:
X_test_final = [sent2features(s) for s in test_prep]
y_test_final = [sent2labels(s) for s in test_prep]

In [0]:
y_pred_final = crf.predict(X_test_final)

In [42]:
test['Label'] =  pd.DataFrame([' '.join(x) for x in y_pred_final])
test.head()


,Subject,Sentence,Label,POS
0,biology,26 . Living things are highly organized and st...,O O B-Definition I-Definition I-Definition I-D...,NUM PUNCT VERB NOUN VERB ADV ADJ CCONJ ADJ PUN...
1,biology,The atom is the smallest and most fundamental ...,O O O B-Definition I-Definition I-Definition I...,DET NOUN VERB DET ADJ CCONJ ADV ADJ NOUN ADP N...
2,biology,It consists of a nucleus surrounded by electro...,B-Referential-Term B-Definition I-Definition I...,PRON VERB ADP DET NOUN VERB ADP NOUN PUNCT
3,biology,"47 . At its most fundamental level , life is m...",O O O O O B-Term I-Term O O O B-Definition I-D...,NUM PUNCT ADP DET ADV ADJ NOUN PUNCT NOUN VERB...
4,biology,Matter is any substance that occupies space an...,B-Term O B-Definition I-Definition I-Definitio...,PROPN VERB DET NOUN DET VERB NOUN CCONJ VERB N...


In [0]:
test[['Sentence','Label']].to_csv('submit_task2_test.csv', index=False)

In [0]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.545     0.300     0.387        20
            I-Alias-Term      0.143     0.040     0.062        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.638     0.576     0.606       288
            I-Definition      0.796     0.877     0.835      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=200,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=200, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.000     0.000     0.000        20
            I-Alias-Term      0.000     0.000     0.000        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.000     0.000     0.000       288
            I-Definition      0.536     0.549     0.542      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

y_pred = crf.predict(X_test)
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.000     0.000     0.000        20
            I-Alias-Term      0.000     0.000     0.000        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.000     0.000     0.000       288
            I-Definition      0.527     0.579     0.552      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
eli5.show_weights(crf, top=10, targets=['O', 'B-ORG', 'I-ORG'])

In [0]:
expl = eli5.explain_weights(crf, top=5, targets=['O', 'B-LOC', 'I-LOC'])
print(eli5.format_as_text(expl))

In [0]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=5,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed: 42.1min
[Parallel(n_jobs=5)]: Done 150 out of 150 | elapsed: 149.0min finished


CPU times: user 58min 56s, sys: 34.5 s, total: 59min 30s
Wall time: 2h 30min 30s


In [0]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 1.6483342862953896, 'c2': 0.12940114154937338}
best CV score: 0.7180427900104681
model size: 0.61M


In [0]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                          precision    recall  f1-score   support

         B-Alias-Te-frag      0.000     0.000     0.000         0
            B-Alias-Term      0.700     0.350     0.467        20
            I-Alias-Term      1.000     0.040     0.077        25
         B-Definiti-frag      0.000     0.000     0.000         1
         I-Definiti-frag      0.000     0.000     0.000         2
            B-Definition      0.619     0.552     0.583       288
            I-Definition      0.748     0.891     0.813      4114
             B-Qualifier      0.000     0.000     0.000         3
             I-Qualifier      0.000     0.000     0.000        22
B-Referential-Definition      0.000     0.000     0.000         2
I-Referential-Definition      0.000     0.000     0.000         2
      B-Referential-Term      0.000     0.000     0.000         3
      I-Referential-Term      0.000     0.000     0.000         1
  B-Secondary-Definition      0.000     0.000     0.000         2
  I-Secon

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [0]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])